In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
data = pd.read_csv('./data/ktw_modified.csv') #wczytanie danych

In [3]:
data.set_index(keys=['Rok','Miesiac','Dzien'], inplace = True) #ustawienie multiindeksu na rok-miesiac-dzien

In [4]:
data.sort_index(inplace = True) #sortowanie po indeksie

In [5]:
data['Typ_opadu'] = data['Typ_opadu'].round(0).astype(int) #zaokrąglenie i rzutowanie na int

In [6]:
data #wyswietlenie gotowych danych

Tmax  Tmin  Tmin_grunt  Opad  Typ_opadu
Rok  Miesiac Dzien                                         
2001 1       1      -0.7  -7.9       -11.1   0.0          1
             2       2.4  -8.7        -8.1   0.1          1
             3       2.2  -3.4        -5.1   5.6          1
             4       6.1   0.5        -2.7   0.0          1
             5       6.5  -1.9        -2.8   0.2          1
...                  ...   ...         ...   ...        ...
2019 12      27      2.7   1.4         0.6   0.6          2
             28      1.5  -1.6        -1.6   0.3          2
             29     -0.1  -1.8        -1.8   0.0          2
             30      3.1  -4.9        -6.1   0.0          2
             31      4.8   2.0         0.4   0.0          0

[6939 rows x 5 columns]

In [7]:
### METODA ŚREDNIEJ I ODCHYLENIA STANDARDOWEGO ###

In [8]:
tmin_std = data.Tmin.std()
tmax_std = data.Tmax.std()

In [9]:
tmin_srednia = data.Tmin.mean()
tmax_srednia = data.Tmax.mean()

In [10]:
tmin_sigma3 = tmin_srednia-(3*tmin_std)
tmax_sigma3 = tmax_srednia+(3*tmax_std)
#z wykresów wiemy że outliery będą jako minimalna tmin więc bierzemy dół przedziału z reguły 3 sigma
#mogą też być jako maksymalna tmax więc wtedy tylko górę przedziału 3 sigma

In [11]:
tmin_sigma3 #jako odchylenia zaklasyfikowane zostały wszystkie temperatury poniżej -18.2*C

-18.262784533256124

In [12]:
tmax_sigma3 #jako odchylenia zaklasyfikowane zostały wszystkie temperatury powyżej 42.9*C

42.98853640289216

In [13]:
tmin_warunek_met1 = data.Tmin < tmin_sigma3
tmax_warunek_met1 = data.Tmax > tmax_sigma3

In [14]:
data.loc[tmin_warunek_met1,'Tmin'] 

Rok   Miesiac  Dzien
2002  1        4       -23.8
               5       -19.1
      12       26      -19.9
2003  1        9       -20.6
      2        2       -18.5
               13      -20.3
2005  2        9       -18.9
2006  1        22      -20.2
               23      -26.8
               24      -24.9
               25      -20.9
               27      -18.3
               28      -18.4
      2        5       -19.7
2009  2        22      -18.5
2010  1        23      -19.6
               24      -21.4
               25      -20.0
               27      -21.2
      12       5       -20.7
               17      -19.1
2012  2        2       -20.2
               3       -23.3
               4       -22.1
               10      -18.7
               11      -22.6
               12      -22.4
               13      -18.5
2017  1        7       -26.0
               8       -24.7
               9       -21.0
               11      -18.9
Name: Tmin, dtype: float64

In [15]:
data.Tmin[tmin_warunek_met1].count() # 32 pomiarów jest outlierami

32

In [16]:
data.loc[tmax_warunek_met1,'Tmax'] 

Series([], Name: Tmax, dtype: float64)

In [17]:
data.Tmax[tmax_warunek_met1].count() # nie ma outlierów w tmax dla 3 sigma

0

In [18]:
#teraz zamiast analizować osobno tmax i tmin łączymy i analizujemy wszystkie temperatury razem

In [19]:
tmax = pd.Series(data.Tmax)
tmin = pd.Series(data.Tmin)

In [20]:
tfull = tmax.append(tmin, ignore_index=False)

In [21]:
tfull

Rok   Miesiac  Dzien
2001  1        1       -0.7
               2        2.4
               3        2.2
               4        6.1
               5        6.5
                       ... 
2019  12       27       1.4
               28      -1.6
               29      -1.8
               30      -4.9
               31       2.0
Length: 13878, dtype: float64

In [22]:
tfull_std = tfull.std()
tfull_srednia = tfull.mean()
tfull_sigma3_dol = tfull_srednia-(3*tfull_std)
tfull_sigma3_gora = tfull_srednia+(3*tfull_std)

In [23]:
tfull_sigma3_dol #jako odchylenia zaklasyfikowane zostały wszystkie temperatury poniżej -20.3*C

-20.38808877560338

In [24]:
tfull_sigma3_gora #jako odchylenia zaklasyfikowane zostały wszystkie temperatury powyżej 38.9*C

38.920125092075494

In [25]:
tfull_waruneka_met1 = tfull < tfull_sigma3_dol
tfull_warunekb_met1 = tfull > tfull_sigma3_gora

In [26]:
tfull[tfull_waruneka_met1 | tfull_warunekb_met1] #dalej znalazło tylko odchylenia dla tmin

Rok   Miesiac  Dzien
2002  1        4       -23.8
2003  1        9       -20.6
2006  1        23      -26.8
               24      -24.9
               25      -20.9
2010  1        24      -21.4
               27      -21.2
      12       5       -20.7
2012  2        3       -23.3
               4       -22.1
               11      -22.6
               12      -22.4
2017  1        7       -26.0
               8       -24.7
               9       -21.0
dtype: float64

In [27]:
### METODA IQR ###

In [28]:
tmin_Q1 = data.Tmin.quantile(0.25)
tmax_Q1 = data.Tmax.quantile(0.25)

In [29]:
tmin_Q3 = data.Tmin.quantile(0.75)
tmax_Q3 = data.Tmax.quantile(0.75)

In [30]:
tmin_IQR = tmin_Q3-tmin_Q1
tmax_IQR = tmax_Q3-tmax_Q1

In [31]:
dol = tmin_Q1-(1.5*tmin_IQR) #tak samo jak poprzednio bierzemy dół przedziału; k=1.5
gora = tmax_Q3+(1.5*tmax_IQR) #sprawdzamy też górę tmax

In [32]:
tmin_warunek_met2 = data.Tmin < dol
tmax_warunek_met2 = data.Tmax > gora

In [33]:
dol #jako odchylenia zaklasyfikowane zostały wszystkie temperatury poniżej -17.7*C

-17.799999999999997

In [34]:
gora #jako odchylenia zaklasyfikowane zostały wszystkie temperatury powyżej -45.5*C

45.75

In [35]:
data.loc[tmin_warunek_met2,'Tmin'] 

Rok   Miesiac  Dzien
2001  12       13      -18.0
2002  1        3       -18.0
               4       -23.8
               5       -19.1
      12       26      -19.9
2003  1        9       -20.6
      2        2       -18.5
               13      -20.3
2005  2        8       -18.1
               9       -18.9
2006  1        22      -20.2
               23      -26.8
               24      -24.9
               25      -20.9
               27      -18.3
               28      -18.4
      2        5       -19.7
               6       -17.9
2009  2        22      -18.5
2010  1        23      -19.6
               24      -21.4
               25      -20.0
               26      -18.0
               27      -21.2
      12       5       -20.7
               16      -17.9
               17      -19.1
2012  2        2       -20.2
               3       -23.3
               4       -22.1
               10      -18.7
               11      -22.6
               12      -22.4
               13     

In [36]:
data.Tmin[tmin_warunek_met2].count() #39 pomiarów jest outlierami

39

In [37]:
data.loc[tmax_warunek_met2,'Tmax']

Series([], Name: Tmax, dtype: float64)

In [38]:
data.loc[tmax_warunek_met2,'Tmax'].count()

0

In [39]:
#teraz zamiast analizować osobno tmax i tmin łączymy i analizujemy wszystkie temperatury razem

In [40]:
tfull_Q1 = tfull.quantile(0.25)

In [41]:
tfull_Q3 = tfull.quantile(0.75)

In [42]:
tfull_IQR = tfull_Q3-tfull_Q1

In [43]:
fdol = tfull_Q1-(1.5*tfull_IQR)
fgora = tfull_Q3+(1.5*tfull_IQR)

In [44]:
fdol

-19.650000000000002

In [45]:
fgora

37.550000000000004

In [46]:
tfull_waruneka_met2 = tfull < fdol
tfull_warunekb_met2 = tfull > fgora

In [47]:
tfull[tfull_waruneka_met2 | tfull_warunekb_met2]

Rok   Miesiac  Dzien
2002  1        4       -23.8
      12       26      -19.9
2003  1        9       -20.6
      2        13      -20.3
2006  1        22      -20.2
               23      -26.8
               24      -24.9
               25      -20.9
      2        5       -19.7
2010  1        24      -21.4
               25      -20.0
               27      -21.2
      12       5       -20.7
2012  2        2       -20.2
               3       -23.3
               4       -22.1
               11      -22.6
               12      -22.4
2017  1        7       -26.0
               8       -24.7
               9       -21.0
dtype: float64

In [48]:
tfull[tfull_waruneka_met2 | tfull_warunekb_met2].count()

21